# More Class Inheritance

Previously, we created a Process abstract base class, and subclassed it with BoundedLinearProcess.

In [6]:
class Process:
    def __init__(self, start_value = 0):
        self.value = start_value
        
    def time_step(self):
        pass
    
    def __str__(self):
        return "Process with current value " + str(self.value)

In [36]:
class BoundedLinearProcess(Process):
    def __init__(self, start_value = 0, velocity = 0):
        super().__init__(start_value)
        self.velocity = velocity
    
    def time_step(self):
        self.value += self.velocity
        if self.value < 0:
            self.value = -self.value
            self.velocity = -self.velocity
        if self.value > 1:
            self.value = 1 - (self.value - 1)
            self.velocity = -self.velocity
        super().time_step

    def __str__(self):
        return " " * int(self.value*20) + "*"

Let's make another process we can play around with.  An autoregressive process of order 1, also called an AR(1) process, is one in which the value at time t is given by, $x_t = \alpha x_{t-1} + w_t$ where $\alpha$ is a constant, $x_{t-1}$ is the previous value, and $w_t$ is a white noise term that is drawn from a normal distribution with standard deviation $\sigma$.

In [39]:
import numpy as np

class ARProcess(Process):
    
    def __init__(self, alpha = 0.5, sigma = 1, start_value = 0):
        super().__init__(start_value)
        self.alpha = alpha
        self.sigma = sigma
        
    def time_step(self):
        self.value = self.alpha * self.value + np.random.normal(scale = self.sigma)
        super().time_step()
        
    def __str__(self):
        if self.value<0:
            s = " " * int(5 * (self.value + 3)) + "*" + " " * int(-self.value * 5) + "|"
        elif self.value== 0:
            s = " " * 15 + "*"
        else:
            s = " " * 15 + "|" + " " * int(5 * self.value) + "*"
        return s

In [30]:
p2 = ARProcess()
for i in range(20):
    print(p2)
    p2.time_step()

               *
           *   |
               |    *
               | *
           *   |
         *     |
               |  *
          *    |
        *      |
              *|
               |        *
               |      *
               |          *
            *  |
            *  |
               |     *
               |    *
          *    |
               | *
               |  *


Let's increase our AR term, alpha, to 0.9 and see the effect on the process.  This new process should show more persistence.  A high value tends to be followed by more high values and a low value tends to be followed by more low values.

In [ ]:
p2 = ARProcess(alpha = 0.9)
for i in range(20):
    print(p2)
    p2.time_step()

As we've seen, we often want to simulate a process several times and observe the results.  Let's create a method to do this.  Since this is a common task for all Processes, we'll want to put it in the Process class.

In [38]:
class Process:
    def __init__(self, start_value = 0):
        self.value = start_value
        
    def time_step(self):
        pass
    
    def __str__(self):
        return "Process with current value " + str(self.value)
    
    def simulate(self, steps = 20):
        for i in range(steps):
            print(self)
            self.time_step()

In [40]:
p1 = BoundedLinearProcess(0,.1)
p2 = ARProcess(alpha = 0.9)

In [42]:
p1.simulate()

*
 *
   *
     *
       *
         *
           *
             *
               *
                 *
                   *
                  *
                *
              *
            *
          *
        *
      *
    *
  *


In [43]:
p2.simulate()

               *
         *     |
          *    |
             * |
        *      |
               |*
          *    |
               |  *
               | *
               |   *
               |           *
               |          *
               |    *
               |        *
               |     *
               |        *
               |   *
               |        *
               |             *
               |             *


Notice that our simulate function works with both child classes.  In one case, self refers to a BoundedLinearProcess, in the other, to an ARProcess.  This is an example of what we call duck typing. Python doesn't care what class self refers to.  All that matters is that self has a time_step() method so that the simulate function can run.

Let's make one more subclass to show how easy it is to reuse code with class inheritance.  A random walk process is a process of the form $x_t = x_{t-1} + w_t$.  Here, $w_t$ is again a white noise term drawn from a normal distribution with standard deviation $sigma$.

Statistically speaking, a random walk is an example of what we call a non-stationary process.  A statistician would say that it is not an autoregressive process, but we can implement it by creating an AR(1) process with attribute `alpha = 1`.

In [45]:
class RandomWalk(ARProcess):
    def __init__(self, sigma = .5):
        super().__init__(alpha = 1, sigma = sigma)

In [46]:
p3 = RandomWalk()
p3.simulate()

               *
               |  *
               |          *
               |           *
               |           *
               |               *
               |                *
               |                   *
               |                 *
               |              *
               |              *
               |              *
               |             *
               |            *
               |            *
               |               *
               |                  *
               |              *
               |                *
               |                *


The random walk process has no tendency to revert to any fixed mean.  It will eventually wander too far away from zero and our nice print statement won't work anymore.  Notice how easy it was to create this new class, utilizing all the machinery found in its parent classes.